# Import et fonctions utilitaires

In [118]:
from PC import PC
import pyAgrum as gum
import pyAgrum.lib.notebook as gnb
import time
import itertools

def get_approxBN(bn,AlgoPC:PC):
    """
    - Si PC rend un graphe mixte (classe d'équivalence de markov ): construit une liste de BN possible
    - Si PC rend un DAG : construit le BN avec la même structure

    Parameters
    ----------
    AlgoPC : PC
        l'objet associé à l'apprentissage

    Returns
    -------
    liste de pyAgrum.BayesNet ou un pyAgrum.BayesNet
        le/les BN associé
    """
    nb_BN=2**len(AlgoPC.G.edges()) #le nombre de BN possible dépend du nombre d'arrete dans la classe d'équivalent de Markov
    bn_approx_base=gum.BayesNet('bn_approx_base')
    allApproxBn=[]
    for nodeID in AlgoPC.G.nodes():
        bn_approx_base.add(gum.LabelizedVariable(bn.variable(nodeID).name(),''))
    for (node1,node2) in AlgoPC.G.arcs():
        bn_approx_base.addArc(bn.variable(node1).name(),bn.variable(node2).name())
    if(len(AlgoPC.G.edges())!=0): #Si G a des arêtes
        allApproxBn=[gum.BayesNet(bn_approx_base) for _ in range(nb_BN)]
        permutations=[]
        for node1,node2 in AlgoPC.G.edges():
            permutations+=list(itertools.permutations((node1,node2))) 
        combinations=list(itertools.combinations(permutations,len(AlgoPC.G.edges())))
        l=[]
        for comb in combinations:
            hasRemoved=False
            for i in range(0,len(comb)-1):
                for j in range(i+1,len(comb)):
                    if( set(comb[i])==set(comb[j])):
                        l.append(comb)
                        hasRemoved=True
                        break
                if(hasRemoved):
                    break
        for comb in l:
            combinations.remove(comb)
        for (bn_i,comb) in zip(allApproxBn,combinations):
            for node1,node2 in comb:
                bn_i.addArc(bn.variable(node1).name(),bn.variable(node2).name())
    else : #Si G est un DAG
        allApproxBn.append(bn_approx_base)
    return allApproxBn

# Algorithme PC

In [122]:
#Générer un BN aléatoire
n=20000
generator=gum.BNGenerator()
bn=generator.generate(n_nodes=5, n_arcs=8, n_modmax=4)
gum.generateCSV(bn,name_out="test.csv", n=20000, show_progress=False, with_labels=False)
#Fonction d'apprentissage avec PC
def learn_PC(n,bn,csvFilePath="test.csv",verbose=False):
    """Approxime un BN à partir de particules données à travers un CSV donné.

    Parameters
    ----------
    csvFilePath : str, optional
        chemin vers le fichier CSV, by default "test.csv"
    verbose : bool, optional
        Permet d'afficher le graphe mixte au cours de l'apprentissage et le BN approximé à la fin avec le BN utilisé pour générer les particules, by default False

    Returns
    -------
    pyAgrum.BayesNet
        Une approximation du BN qui a généré les particules
    """
    if verbose :
        start=time.time()
    AlgoPC=PC(csvFilePath=csvFilePath)
    if verbose:
        GNonOriente=gum.MixedGraph(AlgoPC.G)
    AlgoPC.phase1()
    if verbose:
        GPhase1=gum.MixedGraph(AlgoPC.G)
    AlgoPC.phase2()
    if verbose:
        end=time.time()
        GPhase2=gum.MixedGraph(AlgoPC.G)
    all_bnApprox=get_approxBN(bn,AlgoPC)
    if verbose:
        duration=end-start
        s='s' if duration>1 else ""
        print(f"Apprentissage en {duration} seconde{s} avec {n} particules généré à partir d'un BN avec {len(bn.nodes())} noeuds, {len(bn.arcs())} arcs et de modalité maximum {bn.maxVarDomainSize()} ")
        print("Graphe 1 : non orienté complet ------ Graphe 2 : après phase 1 ------ Graphe 3 : après phase 2")
        gnb.sideBySide(GNonOriente,GPhase1,GPhase2)
        print("1er BN = BN utilisé pour générer les particules ------ à partie du 2ème BN = Tous les BN possibles d'après l'algorithme PC")
        gnb.sideBySide(bn,*all_bnApprox)
    return all_bnApprox

#Exemple
bn_approx=learn_PC(n,bn,csvFilePath="test.csv",verbose=True)


Apprentissage en 0.036794185638427734 seconde avec 20000 particules généré à partir d'un BN avec 5 noeuds, 6 arcs et de modalité maximum 4 
Graphe 1 : non orienté complet ------ Graphe 2 : après phase 1 ------ Graphe 3 : après phase 2


no_name <!-- 0 --> 0 0 <!-- 1 --> 1 1 <!-- 0->1 --> 0->1 <!-- 2 --> 2 2 <!-- 0->2 --> 0->2 <!-- 3 --> 3 3 <!-- 0->3 --> 0->3 <!-- 4 --> 4 4 <!-- 0->4 --> 0->4 <!-- 1->2 --> 1->2 <!-- 1->3 --> 1->3 <!-- 1->4 --> 1->4 <!-- 2->3 --> 2->3 <!-- 2->4 --> 2->4 <!-- 3->4 --> 3->4,no_name <!-- 0 --> 0 0 <!-- 1 --> 1 1 <!-- 0->1 --> 0->1 <!-- 2 --> 2 2 <!-- 0->2 --> 0->2 <!-- 3 --> 3 3 <!-- 0->3 --> 0->3 <!-- 1->2 --> 1->2 <!-- 4 --> 4 4 <!-- 1->4 --> 1->4 <!-- 2->4 --> 2->4,no_name <!-- 0 --> 0 0 <!-- 2 --> 2 2 <!-- 0->2 --> 0->2 <!-- 1 --> 1 1 <!-- 1->0 --> 1->0 <!-- 1->2 --> 1->2 <!-- 4 --> 4 4 <!-- 1->4 --> 1->4 <!-- 3 --> 3 3 <!-- 3->0 --> 3->0 <!-- 4->2 --> 4->2


1er BN = BN utilisé pour générer les particules ------ à partie du 2ème BN = Tous les BN possibles d'après l'algorithme PC


G <!-- n_3 --> n_3 n_3 <!-- n_0 --> n_0 n_0 <!-- n_3->n_0 --> n_3->n_0 <!-- n_2 --> n_2 n_2 <!-- n_0->n_2 --> n_0->n_2 <!-- n_1 --> n_1 n_1 <!-- n_1->n_0 --> n_1->n_0 <!-- n_1->n_2 --> n_1->n_2 <!-- n_4 --> n_4 n_4 <!-- n_4->n_2 --> n_4->n_2 <!-- n_4->n_1 --> n_4->n_1,G <!-- n_3 --> n_3 n_3 <!-- n_0 --> n_0 n_0 <!-- n_3->n_0 --> n_3->n_0 <!-- n_2 --> n_2 n_2 <!-- n_0->n_2 --> n_0->n_2 <!-- n_1 --> n_1 n_1 <!-- n_1->n_0 --> n_1->n_0 <!-- n_1->n_2 --> n_1->n_2 <!-- n_4 --> n_4 n_4 <!-- n_1->n_4 --> n_1->n_4 <!-- n_4->n_2 --> n_4->n_2,G <!-- n_3 --> n_3 n_3 <!-- n_0 --> n_0 n_0 <!-- n_3->n_0 --> n_3->n_0 <!-- n_2 --> n_2 n_2 <!-- n_0->n_2 --> n_0->n_2 <!-- n_1 --> n_1 n_1 <!-- n_1->n_0 --> n_1->n_0 <!-- n_1->n_2 --> n_1->n_2 <!-- n_4 --> n_4 n_4 <!-- n_4->n_2 --> n_4->n_2 <!-- n_4->n_1 --> n_4->n_1


## Analyse Expérimentale de PC  
### TODO : Lancer 100 apprentissages avec des CSV avec n=[1000,5000,10000,15000,20000]  et des BN de différentes tailles
### TODO : Discuter des résultats .. critères : 
- Qualité de l'apprentissage : Structural Hamming, F-score, dist2opt, etc.  
- Rapidité de l'apprentissage 

# Algorithme PC-Stable

## Analyse Expérimentale de PC-Stable

# Conclusion

# Bonus ?